### Testing MC Env Learning in 2 v 2 environment

#### Constants and loading

In [6]:
import numpy as np
import time
import copy

from vgc.datatypes.Objects import PkmTeam, Pkm, GameState, Weather
from vgc.engine.PkmBattleEnv import PkmBattleEnv
from vgc.util.generator.PkmTeamGenerators import RandomTeamGenerator

from vgc.datatypes.Constants import TYPE_CHART_MULTIPLIER, MAX_HIT_POINTS, MOVE_MAX_PP, DEFAULT_TEAM_SIZE
from vgc.datatypes.Objects import PkmMove, Pkm, PkmTeam, GameState, Weather
from vgc.datatypes.Types import PkmStat, PkmType, WeatherCondition, \
    N_TYPES, N_STATUS, N_STATS, N_ENTRY_HAZARD, N_WEATHER, PkmStatus, PkmEntryHazard


import pprint
import pickle

In [7]:
'''
grab the value from the lookup dict and see if can be used

DRY for the two loops

review flow of all code

reveiw and test all parts of the code

can this be generalized more?
    ie not just for the first move when eval but any move?
    idk, probably not for now

due to the variability in outcomes I think I need a ton of results to tell if swap is better or not
    like 1000 for each state and since like 500,000 states that is 500,000,000 battles

TEST need to turn the states into the dict
    TEST want to store counts and running mean of the reward

TEST set up the evaluation loop for this agent vs. the base always attack agent

TEST need to test how many battles can get through in how much time

TEST saving the action dict

TEST need to select the initial action
TEST need to then to attack later
TEST need to convert the attack action into the best attack
TEST need to convert the swap action into a swap
TEST need to store the result of the battle
TEST convert outcome into a rewards

Later

functionalize the build dict loop
functionalize the eval
can run the dict and eval in python files
can parallelize the dict building

maybe store all states in a list then to the dict
    works I think as long as all actions past that point are attacks
    could then combine that later with a swap at that point maybe?
        idk maybe not... could be the 2nd swap and then things are necessarily clear
            ie initial pkm has been revealed and may have taken dmg (though maybe that doesn't matter)

possibly store the state dict attack action for non first actions as well

            
need to add the hiding part
    i guess it's more like some states don't know opp dmg to current pkm and sometimes do

    maybe parallelize if any of this works


can I get the best dmg from both teams even if the pkm stuff is hidden and not revealed?
    probably yes since passing in the team specific state

can check to see how accurate the attack function is
'''

"\n\nneed to turn the states into the dict\n    TEST want to store counts and running mean of the reward\n\nset up the evaluation loop for this agent vs. the base always attack agent\n\nTEST need to test how many battles can get through in how much time\n\nTEST saving the action dict\n\nTEST need to select the initial action\nTEST need to then to attack later\nTEST need to convert the attack action into the best attack\nTEST need to convert the swap action into a swap\nTEST need to store the result of the battle\nTEST convert outcome into a rewards\n\nLater\n\nfunctionalize the build dict loop\nfunctionalize the eval\ncan run the dict and eval in python files\ncan parallelize the dict building\n\nmaybe store all states in a list then to the dict\n    works I think as long as all actions past that point are attacks\n    could then combine that later with a swap at that point maybe?\n        idk maybe not... could be the 2nd swap and then things are necessarily clear\n            ie init

#### Functions for Loops

In [127]:
def turn_agent_action_into_env_action(action, agent_game_state):
        '''
        Action values are
        0: select best move
        1: switch to first pkm
        2: switch to second pkm

        Env actions are
        0 to 3: action of active pokm
        4: switch to first pkm
        5: switch to second pkm
        '''
        # always get best move and action dmg list
        best_active_action, best_damage_list = get_best_active_damage_action(agent_game_state)

        if action == 0:
            # get best dmg action
            action = best_active_action
        else:
            # switch to first or second pkm if alive
            if action == 1 or action == 2:
                pkm = agent_game_state.teams[0].party[action-1]
                if pkm.fainted() or pkm.hp <= 0.0:
                    action = best_active_action
                else:
                    action = action + 3
            else:
                action = best_active_action

        return action, best_damage_list


def get_best_active_damage_action(g: GameState):
    '''
    '''
    # Get weather condition
    weather = g.weather.condition

    # Get my Pokémon team
    my_team = g.teams[0]
    my_pkms = [my_team.active] + my_team.party

    # Get opponent's team
    opp_team = g.teams[1]
    opp_active = opp_team.active

    opp_active_type = opp_active.type
    opp_defense_stage = opp_team.stage[PkmStat.DEFENSE]

    # Iterate over all my Pokémon and their moves to find the most damaging move
    best_dmg_list = []
    best_move_list = []

    for i, pkm in enumerate(my_pkms):
        # Initialize variables for the best move and its damage
        best_damage = -np.inf
        best_move_id = -1

        if i == 0:
            my_attack_stage = my_team.stage[PkmStat.ATTACK]
        else:
            my_attack_stage = 0

        for j, move in enumerate(pkm.moves):
            
            damage = estimate_damage(move.type, pkm.type, move.power, opp_active_type, my_attack_stage,
                                        opp_defense_stage, weather)
            
            # Check if the current move has higher damage than the previous best move
            if damage > best_damage:
                best_move_id = j + i * 4 # think for 2024 j is 0 to 3 for each
                best_damage = damage

        # get best move and dmg for each pokemon
        best_dmg_list.append(best_damage)
        best_move_list.append(best_move_id)

    active_pkm_best_move_id = best_move_list[0]

    if active_pkm_best_move_id < 0 or active_pkm_best_move_id > 3:
        print(f"Error: best move id { active_pkm_best_move_id } not in expected range")
        active_pkm_best_move_id = 0

    return active_pkm_best_move_id, best_dmg_list


def estimate_damage(move_type: PkmType, pkm_type: PkmType, move_power: float, opp_pkm_type: PkmType,
                    attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
        '''
        Not from original code. from updated repo
        '''
        stab = 1.5 if move_type == pkm_type else 1.
        if (move_type == PkmType.WATER and weather == WeatherCondition.RAIN) or (
                move_type == PkmType.FIRE and weather == WeatherCondition.SUNNY):
            weather = 1.5
        elif (move_type == PkmType.WATER and weather == WeatherCondition.SUNNY) or (
                move_type == PkmType.FIRE and weather == WeatherCondition.RAIN):
            weather = .5
        else:
            weather = 1.
        stage_level = attack_stage - defense_stage
        stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / (np.abs(stage_level) + 2.)
        damage = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type] * stab * weather * stage * move_power

        #print(damage, move_type, pkm_type, move_power, opp_pkm_type, attack_stage, defense_stage, weather)
        return damage


def save_object_as_pkl(object_to_save, save_tag):
    '''
    Save object a pickle file
    '''
    with open(f'{save_tag}.pickle', 'wb') as handle:
        pickle.dump(object_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

# make a dict that has keys for 0 to 100 and values for the action dict
def make_lookup_dict():
    lookup_dict = {}
    for i in range(100):
        if i <= 40:
            lookup_value = i // 5
        else:
            lookup_value = 4 + i // 10
        lookup_dict[i] = lookup_value
    return lookup_dict

# lookup_dict = make_lookup_dict()
# pprint.pprint(lookup_dict)

def get_win_loss_reward(terminated, winner, player_index):
    '''
    Does a reward for winning or losing
    winner is -1 unless a winner has been picked
    '''
    reward = 0.
    if terminated:

        if winner == 0 or winner == 1:
            if winner == player_index:
                reward = 1.
            else:
                reward = -1.
        #print(f"reward {reward} | terminated {terminated} | winner {self.env.winner} | player_index {player_index}|")
    return reward

def get_running_mean(old_mean, old_count, new_value):
    '''
    '''
    new_mean = (old_mean * old_count + new_value) / (old_count + 1)
    
    return new_mean


def add_results_to_action_dict(action_dict, state_key, agent_first_move, win_int):
    '''
    '''
    count_key = "count"
    sum_wins_key = "sum_wins"

    if state_key in action_dict:
        if agent_first_move in action_dict[state_key]:
            action_dict[state_key][agent_first_move][sum_wins_key] += win_int
            action_dict[state_key][agent_first_move][count_key] += 1
        else:
            action_dict[state_key][agent_first_move] = {}
            action_dict[state_key][agent_first_move][sum_wins_key] = win_int
            action_dict[state_key][agent_first_move][count_key] = 1
    else:
        action_dict[state_key] = {}
        action_dict[state_key][agent_first_move] = {}
        action_dict[state_key][agent_first_move][sum_wins_key] = win_int
        action_dict[state_key][agent_first_move][count_key] = 1

def add_action_to_pkm_env_action_dict(env_action, my_dict, team_key):
    if env_action in my_dict[team_key]:
        my_dict[team_key][env_action] += 1
    else:
        my_dict[team_key][env_action] = 1

    return my_dict



# def add_results_to_action_dict(action_dict, state_key, agent_first_move, agent_reward):
#     '''
#     '''
#     if state_key in action_dict:
#         if agent_first_move in action_dict[state_key]:
#             action_dict[state_key][agent_first_move]["avg_reward"] = get_running_mean(action_dict[state_key][agent_first_move]["avg_reward"],
#                                                                                   action_dict[state_key][agent_first_move]["count"], agent_reward)
#             action_dict[state_key][agent_first_move]["count"] += 1
#         else:
#             action_dict[state_key][agent_first_move] = {}
#             action_dict[state_key][agent_first_move]["avg_reward"] = agent_reward
#             action_dict[state_key][agent_first_move]["count"] = 1

# a = (1, 2, 3)
# type(a)
# # combine two tuples
# b = a + (4, 5, 6)
# b
# # append the value 7 to the tuple
# b = b + (7,)
# b


def get_hp_array(game_state_agent, game_state_opp):
    '''
    '''
    agent_pkm_hp_list = [game_state_agent.teams[0].active.hp]

    for pkm in game_state_agent.teams[0].party:
        agent_pkm_hp_list.append(pkm.hp)

    opp_active_pkm_hp = game_state_opp.teams[0].active.hp

    hp_array = np.array(agent_pkm_hp_list + [opp_active_pkm_hp])

    return hp_array

def turn_game_state_into_dict_key(game_state_agent, game_state_opp,lookup_dict,
    dmg_array,                      
    pkm_hp_max = 480., dmg_scale_value = 600.):
    '''
    tuple is (
        # HP
        agent_active_pkm_hp, agent_party+_pkm_hp, opp_active_pkm_hp,
        # DMG to opp
        agent_active_pkm_dmg, agent_party_pkm_dmg,
        # dmg from opp
        # do this later
        )
    
    If everything is on the scale of 0 to 100 picturing
    8 buckets from 0 to 40 with increments of 5
    6 buckets from 40 to 100 with increments of 10

    preload a dict with the look up, then scal everything here

    scaling dmg more than max hp so if move can do over 480 dmg has some sort of knowledge ofit
    '''
    # get arrays to make tuples out of for dict key
    hp_array = get_hp_array(game_state_agent, game_state_opp)

    hp_tuple = scale_hp_and_get_dict_value(hp_array, pkm_hp_max, lookup_dict)
    dmg_tuple = scale_hp_and_get_dict_value(dmg_array, dmg_scale_value, lookup_dict)

    dict_key = hp_tuple + dmg_tuple

    return dict_key


def scale_hp_and_get_dict_value(hp_array, max_hp, lookup_dict):
    '''
    '''

    # scale by max hp then multiply by 100 to get into 0 to 99 range
    hp_array = (hp_array / max_hp) * 100
    # round, convert to int then clip to 0 to 99
    hp_array = hp_array.round(0).astype(int).clip(0, 99)

    

    # for hp in hp_array:
    #     hp_values_from_dict_tuple = hp_values_from_dict_tuple + (lookup_dict[hp],)

    hp_values_from_dict_tuple = tuple(lookup_dict[hp] for hp in hp_array)

    return hp_values_from_dict_tuple


    

In [114]:
# lookup_dict = make_lookup_dict()

# a = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)

# print(a)

# a = (1, 2, 3)
# type(a)
# # combine two tuples
# b = a + (4, 5, 6)
# b
# print(b)
# print(type(b))

#### Build Dict Loop

In [130]:
num_battles = 100

winner_dict = {
    0:0,
    1:0,
    -1:0
}

pkm_env_action_dict = {
    0:{},
    1:{},
}

action_state_results_dict = {}

max_episode_steps = 250
agent_index = 0

lookup_dict = make_lookup_dict()
team_generator = RandomTeamGenerator(2)

time_int = int(time.time())
save_tag =  f"_smoke_test_{time_int}"
start_time = time.time()


for battle_idx in range(num_battles):
    
    agent_team = team_generator.get_team().get_battle_team([0, 1, ])
    opp_team = team_generator.get_team().get_battle_team([0, 1, ])

    # set new environment with teams
    env = PkmBattleEnv((agent_team, opp_team),
                   encode=(False, False)) 

    game_state, info = env.reset()

    is_first_move = True
    agent_first_move = None
    state_key = None

    for episode_step in range(max_episode_steps):
        if is_first_move:
            if np.random.rand() < 0.5:
                agent_pre_env_action = 0
                agent_first_move = 'attack'
            else:
                agent_pre_env_action = 1
                agent_first_move = 'swap'
            is_first_move = False
        else:
            agent_pre_env_action = 0

        agent_env_action, agent_team_best_damage_list = turn_agent_action_into_env_action(agent_pre_env_action, game_state[0])
        opp_action, opp_best_damage_list = get_best_active_damage_action(game_state[1])

        if agent_pre_env_action == 1 and agent_env_action != 4:
            print("Error agent action is 1 but env action is not 4 ")
        elif agent_pre_env_action == 0:
            if (agent_env_action < 0 or agent_env_action > 3):
                print("Error agent action is 0 but env action is not 0 to 3 ")
            elif len(agent_team_best_damage_list) == 0:
                print("Error agent action is 0 but best damage list is empty")
            elif agent_team_best_damage_list[0] < 0:
                print("Error agent action is 0 but best damage is negative")

        if opp_action < 0 or opp_action > 3:
            print("Error opp action is not 0 to 3")
        elif len(opp_best_damage_list) == 0:
            print("Error opp best damage list is empty")
        elif opp_best_damage_list[0] < 0:
            print("Error opp best damage is negative")
        
        # get the state key
        # only do this on the initial set up of the env
        if state_key is None:
            # for now just doing part dmg to opp
            #dmg_array = np.array(agent_best_damage_list + [opp_best_damage,])
            dmg_array = np.array(agent_team_best_damage_list)
            state_key = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)
            if len(state_key) != 5:
                print("Error state key is not 5 long")

        # enter action and step the env
        action_list = [agent_env_action, opp_action]
        game_state, _not_used_reward, terminated, truncated, info = env.step(action_list)  # for inference, we don't need reward

        if episode_step == max_episode_steps - 1:
            print('Warning: max steps reached')
            terminated = True

        if terminated:
            winner = env.winner
            if winner == agent_index:
                win_int = 1
            else:
                win_int = 0

            add_results_to_action_dict(action_state_results_dict, state_key, agent_first_move, win_int)
            break

end_time = time.time()
print(f"Time to run {(end_time - start_time) / 60:.3f} minutes")
print(f"Time to run {(end_time - start_time) / num_battles:.3f} seconds per battle")
print(f"Time to run {((end_time - start_time) / num_battles / 60 / 60) * 1000000:.3f} hours per million battles")

print(winner_dict)
# print(action_dict)


save_object_as_pkl(action_state_results_dict , 'action_dict_smoke_test')


Time to run 0.010 minutes
Time to run 0.006 seconds per battle
Time to run 1.586 hours per million battles
{0: 0, 1: 0, -1: 0}


array([], dtype=float64)

In [132]:
len(action_state_results_dict)

98

In [134]:
# action_state_results_dict

{(5, 6, 5, 12, 11): {'attack': {'sum_wins': 0, 'count': 1}},
 (6, 8, 6, 12, 9): {'attack': {'sum_wins': 1, 'count': 1}},
 (5, 5, 5, 8, 10): {'attack': {'sum_wins': 0, 'count': 1}},
 (8, 8, 6, 2, 5): {'swap': {'sum_wins': 0, 'count': 1}},
 (8, 9, 8, 4, 6): {'attack': {'sum_wins': 0, 'count': 1}},
 (6, 8, 8, 3, 5): {'swap': {'sum_wins': 0, 'count': 1}},
 (8, 10, 8, 8, 5): {'attack': {'sum_wins': 1, 'count': 1}},
 (5, 5, 8, 8, 7): {'swap': {'sum_wins': 0, 'count': 1}},
 (8, 8, 5, 7, 8): {'attack': {'sum_wins': 1, 'count': 1}},
 (5, 5, 5, 8, 8): {'attack': {'sum_wins': 0, 'count': 1},
  'swap': {'sum_wins': 0, 'count': 1}},
 (5, 6, 5, 9, 8): {'swap': {'sum_wins': 0, 'count': 1}},
 (10, 8, 8, 3, 3): {'attack': {'sum_wins': 0, 'count': 1}},
 (10, 11, 11, 6, 5): {'attack': {'sum_wins': 1, 'count': 1}},
 (5, 6, 5, 4, 13): {'swap': {'sum_wins': 0, 'count': 1}},
 (6, 6, 5, 6, 6): {'swap': {'sum_wins': 0, 'count': 1}},
 (6, 8, 6, 5, 10): {'attack': {'sum_wins': 1, 'count': 1}},
 (5, 6, 5, 8, 6): 

In [95]:
state_key

({8}, {10})

#### Eval Agents Loop
* based on two agents and results, see how statistically signficant the difference is

0

In [91]:
num_battles = 10

winner_dict = {
    0:0,
    1:0,
    -1:0
}

pkm_env_action_dict = {
    0:{},
    1:{},
}

action_lookup_dict = copy.deepcopy(action_state_results_dict)

max_episode_steps = 250
agent_index = 0

lookup_dict = make_lookup_dict()
team_generator = RandomTeamGenerator(2)

time_int = int(time.time())
save_tag =  f"_smoke_test_winner_dict_{time_int}"
start_time = time.time()


for battle_idx in range(num_battles):
    
    agent_team = team_generator.get_team().get_battle_team([0, 1, ])
    opp_team = team_generator.get_team().get_battle_team([0, 1, ])

    # set new environment with teams
    env = PkmBattleEnv((agent_team, opp_team),
                   encode=(False, False)) 
 
    game_state, info = env.reset()

    # is_first_move = True
    # agent_first_move = None
    state_key = None

    for episode_step in range(max_episode_steps):

        agent_pre_env_action = 0
        agent_env_action, agent_team_best_damage_list = turn_agent_action_into_env_action(agent_pre_env_action, game_state[0])
        opp_action, opp_best_damage_list = get_best_active_damage_action(game_state[1])

        if opp_action < 0 or opp_action > 3:
            print("Error opp action is not 0 to 3")
        elif len(opp_best_damage_list) == 0:
            print("Error opp best damage list is empty")
        elif opp_best_damage_list[0] < 0:
            print("Error opp best damage is negative")
            
        if state_key is None:
            # for now just doing part dmg to opp
            #dmg_array = np.array(agent_best_damage_list + [opp_best_damage,])
            dmg_array = np.array(agent_team_best_damage_list)
            state_key = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)

            # for now only override the initial action, could change this to possibly swap later as well
            if state_key in action_lookup_dict:
                agent_env_action = max(action_lookup_dict[state_key], key=action_lookup_dict[state_key].get)


        if agent_pre_env_action == 1 and agent_env_action != 4:
            print("Error agent action is 1 but env action is not 4 ")
        elif agent_pre_env_action == 0:
            if (agent_env_action < 0 or agent_env_action > 3):
                print("Error agent action is 0 but env action is not 0 to 3 ")
            elif len(agent_team_best_damage_list) == 0:
                print("Error agent action is 0 but best damage list is empty")
            elif agent_team_best_damage_list[0] < 0:
                print("Error agent action is 0 but best damage is negative")

        
   
        # enter action and step the env
        action_list = [agent_env_action, opp_action]

        pkm_env_action_dict = add_action_to_pkm_env_action_dict(agent_env_action, pkm_env_action_dict, 0)
        pkm_env_action_dict = add_action_to_pkm_env_action_dict(opp_action, pkm_env_action_dict, 1)

        #print(action_list)
        game_state, _not_used_reward, terminated, truncated, info = env.step(action_list)  # for inference, we don't need reward

        if episode_step == max_episode_steps - 1:
            print('Warning: max steps reached')
            terminated = True

        if terminated:
            winner = env.winner
            if winner == agent_index:
                win_int = 1
            else:
                win_int = 0

            if winner in winner_dict:
                winner_dict[winner] += 1

            break

end_time = time.time()
print(f"Time to run {(end_time - start_time) / 60:.3f} minutes")
print(f"Time to run {(end_time - start_time) / num_battles:.3f} seconds per battle")
print(f"Time to run {((end_time - start_time) / num_battles / 60 / 60) * 1000000:.3f} hours per million battles")

print(winner_dict)
print(pkm_env_action_dict)

save_object_as_pkl(winner_dict, 'eval_winner_dict_smoke_test')

Time to run 0.038 minutes
Time to run 0.005 seconds per battle
Time to run 1.281 hours per million battles
{0: 236, 1: 264, -1: 0}
{0: {3: 311, 1: 316, 0: 510, 2: 314}, 1: {0: 528, 3: 302, 1: 324, 2: 297}}


#### See if results significant
* should probably do this with ELO?

In [92]:
from scipy.stats import chi2_contingency

def chi_square_test(win_loss_draw1, win_loss_draw2):
    """
    Performs a Chi-square test on two entities with win, loss, and draw counts.
    
    Parameters:
    - win_loss_draw1: A list or tuple of win, loss, and draw counts for entity 1 (e.g., [wins, losses, draws]).
    - win_loss_draw2: A list or tuple of win, loss, and draw counts for entity 2.
    
    Returns:
    - Chi-square statistic, p-value, and interpretation as a string.
    """
    # Create a contingency table
    contingency_table = [win_loss_draw1, win_loss_draw2]
    
    # Perform the Chi-square test
    chi2, pval, dof, expected = chi2_contingency(contingency_table)
    
    # Interpret the results
    interpretation = ("There is a statistically significant difference in outcomes between the two entities."
                      if pval < 0.05 else
                      "There is no statistically significant difference in outcomes between the two entities.")
    
    print(f'Player 1 Win rate: { win_loss_draw1[0] / sum(win_loss_draw1):.3f}')


    print(f'Chi-square statistic: {chi2:.3f}')
    print(f'P-value: {pval:.5f}')

    return chi2, pval, interpretation

# # Example usage
#chi2, pval, interpretation = chi_square_test([120, 80, 50], [130, 70, 60])

chi2, pval, interpretation = chi_square_test([winner_dict[0],
                                              winner_dict[1],
                                              #winner_dict[-1]
                                              ],
                                             [winner_dict[1],
                                              winner_dict[0],
                                              #winner_dict[-1]
                                              ],)

Player 1 Win rate: 0.472
Chi-square statistic: 2.916
P-value: 0.08771


In [83]:
# test where it is significant

for x in range(150, 200, 5):
    winner_dict = {
        0:x,
        1:300-x,
        -1:0
    }
    print("--- x is ", x)
    chi2, pval, interpretation = chi_square_test([winner_dict[0],
                                                winner_dict[1],
                                                #winner_dict[-1]
                                                ],
                                                [winner_dict[1],
                                                winner_dict[0],
                                                #winner_dict[-1]
                                                ],)
    print("________")

--- x is  150
Player 1 Win rate: 0.500
Chi-square statistic: 0.000
P-value: 1.00000
________
--- x is  155
Player 1 Win rate: 0.517
Chi-square statistic: 0.540
P-value: 0.46243
________
--- x is  160
Player 1 Win rate: 0.533
Chi-square statistic: 2.407
P-value: 0.12082
________
--- x is  165
Player 1 Win rate: 0.550
Chi-square statistic: 5.607
P-value: 0.01789
________
--- x is  170
Player 1 Win rate: 0.567
Chi-square statistic: 10.140
P-value: 0.00145
________
--- x is  175
Player 1 Win rate: 0.583
Chi-square statistic: 16.007
P-value: 0.00006
________
--- x is  180
Player 1 Win rate: 0.600
Chi-square statistic: 23.207
P-value: 0.00000
________
--- x is  185
Player 1 Win rate: 0.617
Chi-square statistic: 31.740
P-value: 0.00000
________
--- x is  190
Player 1 Win rate: 0.633
Chi-square statistic: 41.607
P-value: 0.00000
________
--- x is  195
Player 1 Win rate: 0.650
Chi-square statistic: 52.807
P-value: 0.00000
________


In [68]:
from scipy.stats import ttest_ind


def compare_rates(successes_group1, observations_group1, successes_group2, observations_group2):
    """
    Compares two observed rates using a two-sample t-test and prints the t-statistic and p-value.
    
    Parameters:
    - successes_group1: Number of successes in group 1
    - observations_group1: Number of observations in group 1
    - successes_group2: Number of successes in group 2
    - observations_group2: Number of observations in group 2
    """
    # Calculate rates
    rate_group1 = successes_group1 / observations_group1
    rate_group2 = successes_group2 / observations_group2

    # Convert rates to "success" arrays
    data_group1 = np.array([1] * successes_group1 + [0] * (observations_group1 - successes_group1))
    data_group2 = np.array([1] * successes_group2 + [0] * (observations_group2 - successes_group2))

    # Perform the two-sample t-test
    stat, pval = ttest_ind(data_group1, data_group2)

    # Print rounded t-statistic and p-value
    print("Rates: {:.3f} vs. {:.3f}".format(rate_group1, rate_group2))
    print(f'T-statistic: {stat:.5f}')
    print(f'P-value: {pval:.5f}')

# Example usage
#compare_rates(45, 100, 50, 120)

compare_rates(winner_dict[0], num_battles, winner_dict[1], num_battles)

Rates: 0.540 vs. 0.460
T-statistic: 1.96261
P-value: 0.05015


In [70]:
for test_wins in [150, 160, 170, 175, 200, 225, 250, 275]:
    print(test_wins)
    compare_rates(test_wins, num_battles, num_battles-test_wins, num_battles)

150
Rates: 0.500 vs. 0.500
T-statistic: 0.00000
P-value: 1.00000
160
Rates: 0.533 vs. 0.467
T-statistic: 1.63390
P-value: 0.10281
170
Rates: 0.567 vs. 0.433
T-statistic: 3.28991
P-value: 0.00106
175
Rates: 0.583 vs. 0.417
T-statistic: 4.13349
P-value: 0.00004
200
Rates: 0.667 vs. 0.333
T-statistic: 8.64581
P-value: 0.00000
225
Rates: 0.750 vs. 0.250
T-statistic: 14.11855
P-value: 0.00000
250
Rates: 0.833 vs. 0.167
T-statistic: 21.87236
P-value: 0.00000
275
Rates: 0.917 vs. 0.083
T-statistic: 36.86585
P-value: 0.00000


In [36]:
print(episode_step)

3


In [31]:
env.step(action_list) 

((<vgc.engine.PkmBattleEnv.PkmBattleEnv at 0x17e31fafac0>,
  <vgc.engine.PkmBattleEnv.PkmBattleEnv at 0x17e31fa12e0>),
 [1.0, 1.0],
 True,
 False,
 {})

In [27]:
.266 / 60 * 1000000 /60

73.88888888888889